<a href="https://colab.research.google.com/github/MuhTaufikAziz/202155202052-Muh_Taufik_Aziz/blob/main/amazonFruit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir /content/fruit

In [ ]:
cp /content/drive/MyDrive/DL/archive.zip /content/fruit

In [ ]:
!unzip /content/fruit/archive.zip

Archive:  /content/fruit/archive.zip
  inflating: ds_frutas_am/test/acai/download (9).jpeg  
  inflating: ds_frutas_am/test/acai/images (16).jpeg  
  inflating: ds_frutas_am/test/acai/images (17).jpeg  
  inflating: ds_frutas_am/test/acai/images (2).jpeg  
  inflating: ds_frutas_am/test/acai/images.jpeg  
  inflating: ds_frutas_am/test/cupuacu/download (2).jpeg  
  inflating: ds_frutas_am/test/cupuacu/download (3).jpeg  
  inflating: ds_frutas_am/test/cupuacu/download (4).jpeg  
  inflating: ds_frutas_am/test/cupuacu/download (5).jpeg  
  inflating: ds_frutas_am/test/cupuacu/images (3).jpeg  
  inflating: ds_frutas_am/test/graviola/download (1).jpeg  
  inflating: ds_frutas_am/test/graviola/download (2).jpeg  
  inflating: ds_frutas_am/test/graviola/download (3).jpeg  
  inflating: ds_frutas_am/test/graviola/download (4).jpeg  
  inflating: ds_frutas_am/test/graviola/images (4).jpeg  
  inflating: ds_frutas_am/test/guarana/download (2).jpeg  
  inflating: ds_frutas_am/test/guarana/down

In [ ]:
#general libs
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.python import training
#reading the datasets
im_shape = (250,250)
TRAINING_DIR = '../input/ds_frutas_am/train'
TEST_DIR = '../input/ds_frutas_am/test'
seed = 10
BATCH_SIZE = 16

In [ ]:
#using keras ImageGenerator and flow_from_directory
#subdivision in test/validation
data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
#if you want data augmentation, uncomment and run this cell
data_generator = ImageDataGenerator(validation_split=0.2,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
epochs = 80

#callback to save the best model
callbacks_list = [keras.callbacks.ModelCheckpoint(filepath='model.h5',monitor='val_loss', save_best_only=True, verbose=1),keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,verbose=1)]

#training
history = model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // BATCH_SIZE,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data=validation_generator,
        verbose = 1,
        validation_steps=nb_validation_samples // BATCH_SIZE)


In [ ]:
#training curves
import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs_x = range(1, len(loss_values) + 1)
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(epochs_x, loss_values, 'bo', label='Training loss')
plt.plot(epochs_x, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation Loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(2,1,2)
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
plt.plot(epochs_x, acc_values, 'bo', label='Training acc')
plt.plot(epochs_x, val_acc_values, 'b', label='Validation acc')
#plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:
# Load the best saved model
from tensorflow.keras.models import load_model

model = load_model('model.h5')

In [ ]:
# Using the validation dataset
score = model.evaluate_generator(validation_generator)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

In [ ]:
# Using the test dataset
score = model.evaluate_generator(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import itertools

#Plot the confusion matrix. Set Normalize = True/False
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
  
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Some reports
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

#On test dataset
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
target_names = classes

#Confution Matrix
cm = confusion_matrix(test_generator.classes, y_pred)
plot_confusion_matrix(cm, target_names, normalize=False, title='Confusion Matrix')

#Classification Report
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))